<a href="https://colab.research.google.com/github/mibalaguer/ImageProcessingML/blob/main/Miguelangel_Balaguera_Task_1_%26_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Programming exercise - Pytorch & Python Part 1-Data preprocessing**

In [ ]:
import json as js
import pandas as pd
from pandas.io.json import json_normalize

#Upload the json file:
with open("./exercises/exercise-dataset/dataset.json") as file:
  ds = js.load(file)

#dataframe build up containing the relevant info inside the json file:
df = json_normalize(ds, "images" )

df


<ipython-input-122-214d45c73ed6>:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(ds, "images" )


,uri,width,height,annotations
0,000000074092.jpg,332,500,"[{'category_id': 'sports ball', 'bbox': [229, ..."
1,000000086220.jpg,640,428,"[{'category_id': 'bus', 'bbox': [38, 142, 476,..."
2,000000323151.jpg,480,640,"[{'category_id': 'pizza', 'bbox': [0, 240, 450..."
3,000000439773.jpg,640,480,"[{'category_id': 'person', 'bbox': [190, 8, 44..."
4,000000467176.jpg,640,428,"[{'category_id': 'tv', 'bbox': [349, 101, 137,..."


In [ ]:
from PIL import Image

# Function to store the whole batch of images in a data-structured like element, in this case an array:
def image_container(path, id, dataframe):
  images=[]
  for i in range(len(dataframe.index)):
    images.append(Image.open(path+str(dataframe.iloc[i][id])).convert('RGB'))
  return images

In [ ]:
import torch
import torchvision.transforms as transforms

# Function that uses the pytorch library to apply the center-cropping transformation at a specific resolution:
def center_cropping(images, resolution):
  tf = transforms.CenterCrop(resolution)
  cro_img = tf(images)
  return cro_img

In [ ]:
from PIL import Image
import os


images_path = './exercises/exercise-dataset/'
result_path = './results/'
df_column_id = 'uri'
cropping_resolution = 320

os.makedirs('results', exist_ok=True)

# Threshold criterion defined for the annotations:
threshold = 0.5

# Annotation container for a specific image:
an_cont = [a for a in df['annotations']]

# Determination of the annotations to be kept:
kept_an = []
for b in range(len(df.index)):
  kept_an.append([])
  for c in range(len(an_cont[b])):

      bound_box = an_cont[b][c]['bbox']
      x1, y1, width, height = bound_box
      x2, y2 = x1 + width, y1 + height
      an_area = width * height
      int_area = max(0, min(x2, cropping_resolution - max(x1, 0))) * max(0, min(y2, cropping_resolution - max(y1, 0)))
      overlap = int_area / an_area
      if overlap >= threshold:
        kept_an[b].append(an_cont[b][c])

# Center cropping the images at 320x320 pixels and saving them in the specified folder
# and Final step, saving the cropped image and ressulting dataset file:
cropped_dataset ={'images':[]}
for j in range(len(df.index)):
  center_cropping(image_container(images_path,df_column_id,df)[j],
                  cropping_resolution).save(result_path+df.iloc[j][df_column_id])

  cropped_dataset['images'].append({'uri': ds['images'][j]['uri'],'width':320,'height':320,
                   'annotations' : kept_an[j]})


with open(result_path+"/new_dataset.json", 'w') as f:
        js.dump(cropped_dataset, f, indent=4)
